In [1]:
import dd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from getsig import getsig
from scipy.interpolate import interp1d
from scipy.signal import medfilt
#
plt.style.use('helvet2')

#### Select a shotnumber and a time array 

In [3]:
shotnr = 30554
tBegin = 1.0
tEnd = 6.5
dt = 0.1
times = np.arange(tBegin, tEnd+dt, dt)

In [4]:
beta = getsig(shotnr, 'TOT', 'beta_N')
wmhd = getsig(shotnr, 'TOT', 'Wmhd')
taue = getsig(shotnr, 'TOT', 'tau_tot')
h98 = getsig(shotnr, 'TTH', 'H/L-facs')
h98y2_index = 7

/afs/ipp/aug/ads-diags/common/python/lib/dd_20180216.py:117: RuntimeWarning: Warning by DDcsgnl (8.1): no calibration of data
  warnings.warn(text.replace('\x00','').strip(), RuntimeWarning)
/afs/ipp/aug/ads-diags/common/python/lib/dd_20180216.py:117: RuntimeWarning: Warning by DDcsgrp (8.1): No PARAM_SET found
  warnings.warn(text.replace('\x00','').strip(), RuntimeWarning)


In [5]:
nev_shotfile = 'DNE'
nev_signal = 'neDdel_2'
nev_channel = 17
nev_experiment = 'sfp'
medfilt_pts = 21

dne = getsig(shotnr, nev_shotfile, nev_signal, exper=nev_experiment)

In [6]:
interp_beta = interp1d(beta.time, beta.data)
beta_df_entry = interp_beta(times)

interp_wmhd = interp1d(wmhd.time, wmhd.data)
wmhd_df_entry = interp_wmhd(times)

interp_taue = interp1d(taue.time, taue.data)
taue_df_entry = interp_taue(times)

interp_h98 = interp1d(h98.time, h98.data[:,h98y2_index])
h98_df_entry = interp_h98(times)

interp_nev = interp1d(dne.time, medfilt(dne.data[:, nev_channel], medfilt_pts))
nev_df_entry = interp_nev(times)

In [7]:
dict_df = {'time':times, 'nev':nev_df_entry, 'h98y2':h98_df_entry, 'taue':taue_df_entry, 'beta':beta_df_entry, 'wmhd':wmhd_df_entry}
ddf = pd.DataFrame(data=dict_df)
df = ddf[['time','nev','beta','taue','h98y2','wmhd']]

In [8]:
df.head()

,time,nev,beta,taue,h98y2,wmhd
0,1.0,1.938533e+19,0.219025,0.060995,0.377433,70724.625062
1,1.1,4.421253e+19,0.427349,0.076347,0.576177,146936.124257
2,1.2,1.155996e+19,0.905335,0.140809,0.979908,335258.718751
3,1.3,0.000000e+00,1.097278,0.132876,1.062062,416174.395867
4,1.4,0.000000e+00,1.303435,0.114760,1.103102,502025.595322


In [9]:
writer = pd.ExcelWriter('./Support_Files/output_30554.xlsx')
df.to_excel(writer,'Sheet1')

In [10]:
def df_dd(shotnr=30554, tBegin=1.0, tEnd=6.5, dt=0.1,
          dne_data=['DNE','neDdel_2',17,'sfp',21],
          ouput_file=None):
    """Returns a pandas dataframe"""
    times = np.arange(tBegin, tEnd+dt, dt)
    
    beta = getsig(shotnr, 'TOT', 'beta_N')
    wmhd = getsig(shotnr, 'TOT', 'Wmhd')
    taue = getsig(shotnr, 'TOT', 'tau_tot')
    h98 = getsig(shotnr, 'TTH', 'H/L-facs')
    h98y2_index = 7
    
    nev_shotfile = 'DNE'
    nev_signal = 'neDdel_2'
    nev_channel = 17
    nev_experiment = 'sfp'
    medfilt_pts = 21

    #Divertor Spectroscpy data
    nev_shotfile = dne_data[0]
    nev_signal = dne_data[1]
    nev_channel = int(dne_data[2])
    nev_experiment = dne_data[3]
    medfilt_pts = int(dne_data[4])

    dne = getsig(shotnr, nev_shotfile, nev_signal, exper=nev_experiment)
    
    #Interpolations
    interp_beta = interp1d(beta.time, beta.data)
    beta_df_entry = interp_beta(times)

    interp_wmhd = interp1d(wmhd.time, wmhd.data)
    wmhd_df_entry = interp_wmhd(times)

    interp_taue = interp1d(taue.time, taue.data)
    taue_df_entry = interp_taue(times)

    interp_h98 = interp1d(h98.time, h98.data[:,h98y2_index])
    h98_df_entry = interp_h98(times)

    interp_nev = interp1d(dne.time, medfilt(dne.data[:, nev_channel], medfilt_pts))
    nev_df_entry = interp_nev(times)
    
    dict_df = {'time':times, 'nev':nev_df_entry, 'h98y2':h98_df_entry, 'taue':taue_df_entry, 'beta':beta_df_entry, 'wmhd':wmhd_df_entry}
    ddf = pd.DataFrame(data=dict_df)
    df = ddf[['time','nev','beta','taue','h98y2','wmhd']]
    
    #if output_file is not None:
    #    writer = pd.ExcelWriter(output_file)
    #    df.to_excel(writer,'Sheet1')
    
    return df

In [20]:
df_35156 = df_dd(shotnr=35156, tBegin=0.5, tEnd=6.5, dt=0.02, dne_data=['FVL', 'Ne', 18, 'AUGD', 21])
df_35157 = df_dd(shotnr=35157, tBegin=0.5, tEnd=6.5, dt=0.02, dne_data=['FVL', 'Ne', 18, 'AUGD', 21])
df_35158 = df_dd(shotnr=35158, tBegin=0.5, tEnd=6.5, dt=0.02, dne_data=['FVL', 'Ne', 18, 'AUGD', 21])

In [19]:
df_35165 = df_dd(shotnr=35165, tBegin=0.5, tEnd=5.7, dt=0.02, dne_data=['FVL', 'Ne', 18, 'AUGD', 21])

In [ ]:
df_35167 = df_dd(shotnr=35167, tBegin=0.5, tEnd=6.5, dt=0.02, dne_data=['FVL', 'Ne', 18, 'AUGD', 21])
df_35356 = df_dd(shotnr=35356, tBegin=0.5, tEnd=6.5, dt=0.02, dne_data=['FVL', 'Ne', 18, 'AUGD', 21])
df_35358 = df_dd(shotnr=35358, tBegin=0.5, tEnd=6.5, dt=0.02, dne_data=['FVL', 'Ne', 18, 'AUGD', 21])
df_35381 = df_dd(shotnr=35381, tBegin=0.5, tEnd=6.5, dt=0.02, dne_data=['FVL', 'Ne', 18, 'AUGD', 21])

In [21]:
df_35156.to_csv('./Support_Files/df_35156.csv')
df_35157.to_csv('./Support_Files/df_35157.csv')
df_35158.to_csv('./Support_Files/df_35158.csv')
df_35165.to_csv('./Support_Files/df_35165.csv')

In [2]:
from dd_df import get_shot_title

In [12]:
print(get_shot_title(35158))
print(get_shot_title(35157))

\#35158 Power exhaust with mixed impurities                                                                                                                                                                                                                                                                                                                                                                         
\#35157 Power exhaust with mixed impurities


\#35158 Power exhaust with mixed impurities                                                                                                                                                                                                                                                                                                                                                                         
